# Mobility-Station-Finder

## Imports

In [81]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
from matrixconverters.read_ptv import ReadPTVMatrix
from collections import namedtuple
import requests

## Read and process static data

### Paths

In [2]:
path_npvm_zones = r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\Verkehrszonen_Schweiz_NPVM_2017_shp.zip"
path_mobility_stations = r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\mobility-stationen-und-fahrzeuge-schweiz.csv"
path_pt_jrta = r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\140_JRTA_(OEV).mtx"
path_pt_ntr = r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\144_NTR_(OEV).mtx"

### Read NPVM-zones

In [3]:
df_npvm_zones = gpd.read_file(path_npvm_zones).to_crs(4326)

In [4]:
df_npvm_zones.head()

,ID,ID_alt,ID_Gem,N_Gem,stg_type,N_stg_type,ID_KT,N_KT,ID_SL3,N_SL3,ID_Agglo,N_Agglo,ID_AMR,N_AMR,geometry
0,101001,1,1,Aeugst am Albis,1,None,1,ZH,3,Ländlich,261,Zürich,12031,DietikonSchlieren,"POLYGON ((8.47334 47.26128, 8.47334 47.26139, ..."
1,201001,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,DietikonSchlieren,"POLYGON ((8.42224 47.29775, 8.42282 47.29816, ..."
2,201002,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,DietikonSchlieren,"POLYGON ((8.44770 47.26794, 8.44767 47.26782, ..."
3,201003,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,DietikonSchlieren,"POLYGON ((8.43834 47.27714, 8.43814 47.27726, ..."
4,201004,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,DietikonSchlieren,"POLYGON ((8.45000 47.27949, 8.45007 47.27945, ..."


In [5]:
len(df_npvm_zones)

7978

In [6]:
df_npvm_zones[df_npvm_zones.ID == 5301003]

,ID,ID_alt,ID_Gem,N_Gem,stg_type,N_stg_type,ID_KT,N_KT,ID_SL3,N_SL3,ID_Agglo,N_Agglo,ID_AMR,N_AMR,geometry
84,5301003,53,53,Bülach,1,None,1,ZH,1,Städtisch,261,Zürich,12032,Kloten,"POLYGON ((8.53973 47.51618, 8.54007 47.51529, ..."


### Read an process Mobility-stations

In [7]:
df_mobility = pd.read_csv(path_mobility_stations, delimiter=";", encoding="utf8")[["Stationsnummer", "Name", "Standort"]].groupby("Stationsnummer").first().dropna()

In [8]:
df_mobility["lon"] = df_mobility["Standort"].apply(lambda x: x.split(",")[1])
df_mobility["lat"] = df_mobility["Standort"].apply(lambda x: x.split(",")[0])

In [9]:
df_mobility

,Name,Standort,lon,lat
Stationsnummer,,,,
1006,Brugg Bahnhof,"47.48154,8.20942",8.20942,47.48154
1012,Arbon Bahnhof,"47.51032,9.43345",9.43345,47.51032
1019,Basel Vogesenstrasse,"47.5686935,7.5748328",7.5748328,47.5686935
1024,Bellinzona Stazione,"46.1963,9.03017",9.03017,46.1963
1025,Brig Bahnhof,"46.31906,7.99028",7.99028,46.31906
...,...,...,...,...
6007,Adnovum Teststation2,"47.144722,8.434389",8.434389,47.144722
6028,Convadis Teststandort 401115,"47.503635,8.239223",8.239223,47.503635
6029,Convadis Teststandort 400611,"47.50362,8.23921",8.23921,47.50362


In [10]:
gdf_mobility = gpd.GeoDataFrame(df_mobility, geometry=gpd.points_from_xy(df_mobility.lon, df_mobility.lat), crs=4326)

In [11]:
gdf_mobility.head()

,Name,Standort,lon,lat,geometry
Stationsnummer,,,,,
1006,Brugg Bahnhof,"47.48154,8.20942",8.20942,47.48154,POINT (8.20942 47.48154)
1012,Arbon Bahnhof,"47.51032,9.43345",9.43345,47.51032,POINT (9.43345 47.51032)
1019,Basel Vogesenstrasse,"47.5686935,7.5748328",7.5748328,47.5686935,POINT (7.57483 47.56869)
1024,Bellinzona Stazione,"46.1963,9.03017",9.03017,46.1963,POINT (9.03017 46.19630)
1025,Brig Bahnhof,"46.31906,7.99028",7.99028,46.31906,POINT (7.99028 46.31906)


### Assign NPVM-zone to Mobility-Stations

In [97]:
mobility_zone = gpd.sjoin(gdf_mobility, df_npvm_zones)[["Name", "Standort", "geometry", "ID", "N_Gem"]]

In [98]:
mobility_zone

,Name,Standort,geometry,ID,N_Gem
Stationsnummer,,,,,
1006,Brugg Bahnhof,"47.48154,8.20942",POINT (8.20942 47.48154),409501008,Brugg
3215,Brugg Post Neumarkt / Bahnhofstrasse,"47.48216,8.20757",POINT (8.20757 47.48216),409501008,Brugg
1012,Arbon Bahnhof,"47.51032,9.43345",POINT (9.43345 47.51032),440101010,Arbon
1019,Basel Vogesenstrasse,"47.5686935,7.5748328",POINT (7.57483 47.56869),270101028,Basel
1024,Bellinzona Stazione,"46.1963,9.03017",POINT (9.03017 46.19630),500201015,Bellinzona
...,...,...,...,...,...
5989,Sion Rue de Lausanne,"46.230406,7.350719",POINT (7.35072 46.23041),626601007,Sion
5990,Fribourg Bourg / Tilleul,"46.80615,7.1611",POINT (7.16110 46.80615),219601017,Fribourg
6028,Convadis Teststandort 401115,"47.503635,8.239223",POINT (8.23922 47.50364),404401001,Untersiggenthal


In [14]:
mobility_zone.to_csv(r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\Analyse_Mobility_Stationen.csv", sep=";")

### Read PT-skims

In [15]:
skim_jrta = ReadPTVMatrix(path_pt_jrta)

In [16]:
skim_ntr = ReadPTVMatrix(path_pt_ntr)

In [17]:
skim_jrta.values

<bound method Mapping.values of <xarray.ReadPTVMatrix>
Dimensions:       (zone_no: 8718, zone_no2: 8718, origins: 8718,
                   destinations: 8718)
Coordinates:
  * zone_no       (zone_no) int32 101001 201001 201002 ... 910000008 910000009
  * zone_no2      (zone_no2) int32 101001 201001 201002 ... 910000008 910000009
  * origins       (origins) int32 101001 201001 201002 ... 910000008 910000009
  * destinations  (destinations) int32 101001 201001 ... 910000008 910000009
Data variables:
    matrix        (origins, destinations) float64 0.0 70.62 44.87 ... 517.0 0.0
    zone_name     (zone_no) object 'Aeugst am Albis' ... 'Strasbourg'
    zone_names2   (zone_no2) object 'Aeugst am Albis' ... 'Strasbourg'
Attributes:
    fn:                   D:\data\ttools gmbh\Projekte - Dokumente\021_Mobili...
    ZeitVon:              nan
    ZeitBis:              nan
    Faktor:               1.0
    VMAktKennung:         0
    AnzBezeichnerlisten:  1
    roundproc:            1
    diags

In [18]:
skim_jrta.sel(origins=378601001).sel(destinations=35101026).matrix.item()

286.0875843780647

In [19]:
skim_ntr.sel(origins=378601001).sel(destinations=35101026).matrix.item()

3.000000000000001

In [20]:
def get_skim(skim_matrix, from_npvm_zone_id, to_npvm_zone_id):
    return skim_matrix.sel(origins=from_npvm_zone_id).sel(destinations=to_npvm_zone_id).matrix.item()

In [21]:
def get_jrta(from_npvm_zone_id, to_npvm_zone_id):
    return get_skim(skim_jrta, from_npvm_zone_id, to_npvm_zone_id)

In [22]:
def get_ntr(from_npvm_zone_id, to_npvm_zone_id):
    return get_skim(skim_ntr, from_npvm_zone_id, to_npvm_zone_id)

In [23]:
get_jrta(378601001, 35101026)

286.0875843780647

In [24]:
get_ntr(378601001, 35101026)

3.000000000000001

## Execute query

### Define origin and destination and assign NPVM-Zone

In [27]:
orig_point = Point(7.423570, 46.936620)
dest_point = Point(7.695260, 46.828540)

In [28]:
df_orig = gpd.GeoDataFrame({'geometry': [orig_point]}, crs="EPSG:4326")
df_dest = gpd.GeoDataFrame({'geometry': [dest_point]}, crs="EPSG:4326")

In [29]:
orig_zone = gpd.sjoin(df_orig, df_npvm_zones)
dest_zone = gpd.sjoin(df_dest, df_npvm_zones)

In [133]:
orig_zone_id = orig_zone["ID"].item()

### Compute potential Mobility-stations

In [31]:
search_area_orig = gpd.GeoDataFrame(geometry=df_orig.to_crs(2026).buffer(5*1000).to_crs(4326), crs="EPSG:4326")
search_area_dest = gpd.GeoDataFrame(geometry=df_dest.to_crs(2026).buffer(50*1000).to_crs(4326), crs="EPSG:4326")

In [95]:
search_area_orig.head()

,geometry
0,"POLYGON ((7.40836 46.90462, 7.40386 46.90580, ..."


In [93]:
gdf_mobility.head()

,Name,Standort,lon,lat,geometry
Stationsnummer,,,,,
1006,Brugg Bahnhof,"47.48154,8.20942",8.20942,47.48154,POINT (8.20942 47.48154)
1012,Arbon Bahnhof,"47.51032,9.43345",9.43345,47.51032,POINT (9.43345 47.51032)
1019,Basel Vogesenstrasse,"47.5686935,7.5748328",7.5748328,47.5686935,POINT (7.57483 47.56869)
1024,Bellinzona Stazione,"46.1963,9.03017",9.03017,46.1963,POINT (9.03017 46.19630)
1025,Brig Bahnhof,"46.31906,7.99028",7.99028,46.31906,POINT (7.99028 46.31906)


In [94]:
mobility_zone.head()

,Name,Standort,lon,lat,geometry,index_right,ID,ID_alt,ID_Gem,N_Gem,stg_type,N_stg_type,ID_KT,N_KT,ID_SL3,N_SL3,ID_Agglo,N_Agglo,ID_AMR,N_AMR
Stationsnummer,,,,,,,,,,,,,,,,,,,,
1006,Brugg Bahnhof,"47.48154,8.20942",8.20942,47.48154,POINT (8.20942 47.48154),5144,409501008,4095,4095,Brugg,1,None,19,AG,1,Städtisch,4021,Baden  Brugg,12010,Baden
3215,Brugg Post Neumarkt / Bahnhofstrasse,"47.48216,8.20757",8.20757,47.48216,POINT (8.20757 47.48216),5144,409501008,4095,4095,Brugg,1,None,19,AG,1,Städtisch,4021,Baden  Brugg,12010,Baden
1012,Arbon Bahnhof,"47.51032,9.43345",9.43345,47.51032,POINT (9.43345 47.51032),5476,440101010,4401,4401,Arbon,1,None,20,TG,1,Städtisch,4401,Arbon  Rorschach,15050,St. Gallen
1019,Basel Vogesenstrasse,"47.5686935,7.5748328",7.5748328,47.5686935,POINT (7.57483 47.56869),3643,270101028,270105,2701,Basel,1,None,12,BS,1,Städtisch,2701,Basel,8013,Basel
1024,Bellinzona Stazione,"46.1963,9.03017",9.03017,46.1963,POINT (9.03017 46.19630),5722,500201015,5002,5002,Bellinzona,1,None,21,TI,1,Städtisch,5002,Bellinzona,13040,Bellinzona


In [100]:
orig_zone_mobility_stations = gpd.sjoin(mobility_zone, search_area_orig)
dest_zone_mobility_stations = gpd.sjoin(mobility_zone, search_area_dest)

In [101]:
len(orig_zone_mobility_stations)

102

In [102]:
len(dest_zone_mobility_stations)

214

In [103]:
potential_mobility_stations = pd.concat([orig_zone_mobility_stations, dest_zone_mobility_stations]).groupby("Stationsnummer").first()

In [104]:
print(len(potential_mobility_stations))

214


In [109]:
potential_mobility_stations.tail().drop(["index_right"], axis=1)

,Name,Standort,geometry,ID,N_Gem
Stationsnummer,,,,,
5926,Grosshöchstetten Bahnhof,"46.905267,7.635508",POINT (7.63551 46.90527),60801001,Grosshöchstetten
5945,Ramsei Bahnhof,"46.997373,7.710658",POINT (7.71066 46.99737),95501003,Lützelflüh
5963,Bern Huebergass,"46.945529,7.416099",POINT (7.41610 46.94553),35101025,Bern
5967,Ittigen (BE) Bahnhof,"46.973383,7.485135",POINT (7.48513 46.97338),36201010,Ittigen
5985,Laupen Bahnhof,"46.9015364,7.2420877",POINT (7.24209 46.90154),66701003,Laupen


In [158]:
potential_mobility_stations.loc[[1041]]

,Name,Standort,geometry,ID,N_Gem,index_right
Stationsnummer,,,,,,
1041,Burgdorf Steinhof,"47.05301,7.61717",POINT (7.61717 47.05301),40401009,Burgdorf,0


### Get Mobility travel time and distance from ORMS

In [127]:
potential_mobility_stations_list = list(potential_mobility_stations[["geometry"]].to_records())

In [111]:
potential_mobility_stations_list[:3]

[(1027, <POINT (7.5 46.886)>),
 (1040, <POINT (7.619 47.061)>),
 (1041, <POINT (7.617 47.053)>)]

In [112]:
coords_str = "{},{}".format(dest_point.x, dest_point.y)
for pot_mob_st in potential_mobility_stations_list:
    coords_str += ";{},{}".format(pot_mob_st[1].x, pot_mob_st[1].y)
url = "https://router.project-osrm.org/table/v1/driving/{}?destinations=0&annotations=duration,distance".format(coords_str)

In [113]:
res = requests.get(url).json()

In [114]:
distances = res["distances"][1:]
durations = res["durations"][1:]

In [115]:
print(len(distances))
print(len(durations))

214
214


In [159]:
distances[:5]

[[30080.1], [42726.2], [41101.6], [40282.8], [43073.1]]

In [160]:
durations[:5]

[[1790.5], [3099.1], [2915.8], [2245.9], [2490.3]]

### Get pt skims from origin to potential Mobility stations

In [134]:
zone_ids_list = list(potential_mobility_stations[["ID"]].to_records())
jrta_list = [get_jrta(orig_zone_id, x[1]) for x in zone_ids_list]
ntr_list = [get_ntr(orig_zone_id, x[1]) for x in zone_ids_list]

In [154]:
zone_ids_list[:5]

[(1027, 86101006),
 (1040, 40401007),
 (1041, 40401009),
 (1042, 35201002),
 (1073, 58101001)]

In [139]:
print(len(jrta_list))
print(len(ntr_list))

214
214


In [155]:
jrta_list[:5]

[30.41301950841377,
 45.149267496633314,
 52.16144416090363,
 53.67167268336198,
 82.38422138863444]

In [156]:
ntr_list[:5]

[0.0, 1.0, 1.5651697916511602, 2.0, 1.1429780436987096]

TODO:
- Calculate generalized costs

### Visualize situation on map

In [ ]:
from ipyleaflet import Map, GeoData, basemaps, LayersControl
import geopandas
import json


m = Map(center=(52.3,8.0), zoom = 3, basemap= basemaps.Esri.WorldTopoMap)

geo_data = GeoData(geo_dataframe = df_npvm_zones,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Countries')

m.add_layer(geo_data)
m.add_control(LayersControl())

In [ ]:
m.layout.width = '100%'
m.layout.height = '1000px'
m
# m.save("npvm-zonen.html")